# AI as a Judge for Prompt Modification
Use AI as a judge to modify a prompt to navigate it through a 'maze'

In [80]:
import sys
import os
import dspy 

import dspy 
from pydantic import BaseModel, Field, StrictInt, validator
from typing import Literal

from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

Getting keys from environment variables


In [81]:
# lm = dspy.LM(
#     'gpt-4.1', 
#     model_type='chat', 
#     cache=False, 
#     api_key=settings.OPENAI_API_KEY,
#     temperature=0.8     
# )

# dspy.configure(lm=lm)

In [82]:
lm = dspy.LM(
        "openai/gpt-5-mini",
        model_type="responses",
        temperature=1.0,
        max_tokens=16000,
    )

dspy.settings.configure(lm=lm)

In [ ]:
class Widget(BaseModel):
    shape: Literal["round", "square", "triangle"] = dspy.OutputField()
    weightInKg: StrictInt = dspy.OutputField(desc="Must be postive integer")  
    
    @validator("weightInKg")
    def must_be_positive(cls, v):
        if v <= 0:
            raise ValueError("weightInKg must be > 0")
        return v

class WidgetOrder(dspy.Signature):
    """
    Follow the ```order_placement_instructions``` to place an order for a widget. 
    Double check that the created widget meets the instructions exactly.
    ️Go through all the details carefully to ensure accuracy and look at each property and double check the values match the rules
    """

    order_placement_instructions: str = dspy.InputField()

    widget: Widget = dspy.OutputField()
    
class WidgetOrderInstructionCreator(dspy.Signature):
    """
    You are an expert at creating orders for widgets. You use previously provided feedback and widgets order attempts to improve the order placement instructions.
    Create detailed ```order_placement_instructions``` for placing an order for a widget based on the original widget details and instructions.
    Only use the properties of the ```previous_widget``` and the ```order_validator_feedback``` to improve the order placement instructions.
    Keep the instructions as concise as possible and to the point.
    """

    previous_widget: Widget = dspy.InputField(desc="This was the previous widget that was placed in the order and was rejected. If it is None then this is the first attempt to place an order.")
    previous_instructions: str = dspy.InputField(desc="These were the previous order placement instructions that were used to create the previous widget order.")
    order_validator_feedback: str = dspy.InputField(desc="These feedback from the order validator about why the order was rejected, if applicable.")

    order_placement_instructions: str = dspy.OutputField()

################################################################
# Function to place order

def place_order(order: WidgetOrder) -> str:
    """
    Places orders for widgets. Returns 'Valid' if the order is valid, otherwise returns a string with the reason for rejection.

    Return: 
    - str: Returns 'Valid' if the order is valid, otherwise returns a string with the reason for rejection.
    """

    if order.widget.shape == "round" and order.widget.weightInKg <= 5:
        md("**Order rejected**: Round widgets cannot weigh less than 5kg.")
        return "Round widgets cannot weigh less than or equal to 5kg. Try again with a different shape or weight."
    
    # dividor = 4
    # if (order.widget.weightInKg % dividor) != 0:
    #     md(f"**Order rejected**: Widget weight is {order.widget.weightInKg}kg, this must divisible by {dividor}.")
    #     return "Widget weight must be an even number. Try again with a different weight."

    return "Valid"


/tmp/ipykernel_13096/2422319892.py:5: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("weightInKg")


In [84]:
create_order = dspy.Predict(WidgetOrder)
create_order_instructions = dspy.Predict(WidgetOrderInstructionCreator)

################################################################
# Place order 

loopCount = 0

instructions = "Create a round that is 3kg in weight."
widget = None
order_validator_feedback = None

md("**Original instructions**: ", instructions)

while loopCount < 5:
    md("### loopCount: ", loopCount)
    
    order_instructions = create_order_instructions(previous_widget=widget, previous_instructions=instructions, order_validator_feedback=order_validator_feedback)
    instructions = order_instructions.order_placement_instructions
    md("**Order instructions**: ", instructions)
    
    order = create_order(order_placement_instructions=order_validator_feedback)
    md("**Order created**: ", order)
    
    response = place_order(order=order)
    
    if response == "Valid":
        md("### Order placed successfully!")
        break
    
    order_validator_feedback = response

    loopCount += 1

**Original instructions**: Create a round that is 3kg in weight.

### loopCount: 0

**Order instructions**: Place an order for a single "round" widget with a target mass of 3.0 kg. Do not proceed until the following clarifications are provided and confirmed by the requester, because mass alone is insufficient to define manufacturing:  
  
Required confirmations (must be returned before manufacturing):  
1. Shape definition: confirm whether "round" means a sphere or a circular/disc (cylinder) profile.    
2. If disc/cylinder: provide outer diameter and thickness. If sphere: provide diameter.    
3. Material and grade (e.g., aluminium 6061-T6, stainless steel 316, etc.). Material choice determines dimensions needed to meet 3.0 kg.    
4. Quantity required.    
5. Acceptable mass tolerance (e.g., ±% or ± grams).    
6. Surface finish and any surface treatments (e.g., anodize, paint, polish).    
7. Any machining features or holes, threads, or mounting details.    
8. Required dimensional tolerances and inspection/certification (e.g., material cert, weight verification).    
9. Packaging and shipping instructions and required delivery date.  
  
Ordering action after confirmations:  
- Calculate dimensions from chosen material density to achieve 3.0 kg within the agreed tolerance.    
- Produce detailed drawing/specification for customer approval prior to manufacturing.    
- Perform final mass verification and include a weight report with shipment.  
  
If any of the required confirmations are not provided within the next communication, return the request for clarification and do not manufacture.

**Order created**: Prediction(  
    widget=Widget(shape='round', weightInKg=1)  
)

**Order rejected**: Round widgets cannot weigh less than 5kg.

...

**Order rejected**: Widget weight is 6kg, this must divisible by 4.

### loopCount: 4

**Order instructions**: This is the first attempt (previous_widget = None). Update the original instructions to emphasize and enforce the validator feedback: the target mass MUST be an even number. If you select a round widget, the mass MUST be an even number strictly greater than 5.0 kg (i.e., minimum valid round mass = 6.0 kg). Orders with a non-even mass will be rejected.  
  
Provide the following concise confirmations exactly as listed below. Use numeric mass values with one decimal place and even numbers (e.g., 6.0, 8.0, 10.0). If Option B (round) is chosen, do not submit any mass ≤ 5.0.  
  
Required return fields (fill every applicable item):  
1) Chosen correction method: Option A (Non‑round) or Option B (Round: Sphere or Disc/Cylinder — specify subtype).  
2) Final target mass (kg): an even number (format X.0). If Option B, mass ≥ 6.0 kg.  
3) Shape/subtype and exact dimensions:  
   - Option A (non‑round): provide exact geometry + dimensions (length × width × height, wall thickness if hollow) or attach CAD.  
   - Option B — Sphere: exact diameter. Disc/Cylinder: outer diameter and thickness; state solid or hollow and inner diameter if hollow.  
4) Material and grade (e.g., aluminum 6061‑T6).  
5) Quantity (integer).  
6) Acceptable mass tolerance (± grams or ±%).  
7) Surface finish and any heat treatment/coating.  
8) Machining/features required (holes, threads, keyways) with sizes/locations or attach drawing.  
9) Dimensional tolerances required and required inspections/certificates (material cert, weight report).  
10) Packaging and shipping instructions and required delivery date (YYYY‑MM‑DD).  
  
Quick validation reminders (will be applied automatically):  
- Mass must be an even number. Non-even masses will be rejected.  
- If round (Option B), mass must be ≥ 6.0 kg and even.  
- Use format examples: Mass = 6.0 kg (valid), Mass = 5.0 kg (invalid), Mass = 7.0 kg (valid), Mass = 7.5 kg (invalid).  
  
Provide your completed confirmations exactly following the numbered list above.

**Order created**: Prediction(  
    widget=Widget(shape='round', weightInKg=4)  
)

**Order rejected**: Round widgets cannot weigh less than 5kg.